In [1]:
%matplotlib inline
from keras import applications
from keras.datasets import mnist
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils import np_utils
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split
from keras import optimizers

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:962: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:962: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #3
  (fname, cnt))
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# Initial Settings

In [2]:
img_size = 100
_lambda = 0.01

# Load Data

In [3]:
data = np.load('processed_images.npy')
labels = np.load('processed_labels.npy')

print(data.shape)
print(labels.shape)

(2953, 100, 100, 3)
(2953,)


# Train Test Split

In [4]:
X_train, X_test, y_train, y_test = train_test_split(data, labels, train_size = 0.8, random_state = 20)

nb_classes = 2
y_train = np.array(y_train)
y_test = np.array(y_test)
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

print("Training matrix shape", X_train.shape)
print("Testing matrix shape", X_test.shape)

print("Training label shape", Y_train.shape)
print("Testing label shape", Y_test.shape)

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


Training matrix shape (2362, 100, 100, 3)
Testing matrix shape (591, 100, 100, 3)
Training label shape (2362, 2)
Testing label shape (591, 2)


# Google's Inception V3

In [9]:
# from keras.applications.inception_v3 import InceptionV3
# from keras.applications.inception_v3 import preprocess_input
# from keras.applications.inception_v3 import decode_predictions

# model = InceptionV3(weights='imagenet', include_top=True)

96116736/96112376 [==============================] - 60s 1us/step


# VGG16

https://github.com/fchollet/deep-learning-models

In [19]:
# model = applications.VGG16(weights = "imagenet", include_top=False, input_shape = (img_size, img_size, 3))

58892288/58889256 [==============================] - 17s 0us/step


In [ ]:
# top_model = Sequential()
# top_model.add(Flatten(input_shape=model.output_shape[1:]))
# top_model.add(Dense(256, activation='relu'))
# top_model.add(Dropout(0.5))
# top_model.add(Dense(1, activation='sigmoid'))

# # note that it is necessary to start with a fully-trained
# # classifier, including the top classifier,
# # in order to successfully do fine-tuning
# top_model.load_weights(top_model_weights_path)

# # add the model on top of the convolutional base
# model.add(top_model)

# for layer in model.layers[:25]:
#     layer.trainable = False

# # compile the model with a SGD/momentum optimizer
# # and a very slow learning rate.
# model.compile(loss='binary_crossentropy',
#               optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
#               metrics=['accuracy'])

# VGG 14

In [20]:
# model = applications.VGG14(weights = "imagenet", include_top=False, input_shape = (img_size, img_size, 3))

AttributeError: module 'keras.applications' has no attribute 'VGG14'

In [ ]:
mode

# VGG-19

In [5]:
model = applications.VGG19(weights = "imagenet", include_top=False, input_shape = (img_size, img_size, 3))

In [6]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 100, 100, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 100, 100, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 100, 100, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 50, 50, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 50, 50, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 50, 50, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 25, 25, 128)       0         
__________

In [ ]:
## Make last two layers trainable


# Create FC Layer

In [7]:
top_model = Sequential()

top_model.add(Flatten(input_shape=model.output_shape[1:]))
top_model.add(Dense(128, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(2, activation='softmax'))

# top_model.load_weights("first_try.h5")

In [8]:
new_model = Sequential() #new model
for layer in model.layers: 
    new_model.add(layer)
    
new_model.add(top_model) # now this works

In [9]:
for layer in model.layers[:21]:
    layer.trainable = False

In [10]:
adam = optimizers.SGD(lr=1e-4, decay=1e-6, momentum=0.9, nesterov=True)
new_model.compile(loss='categorical_crossentropy',
              optimizer= adam,
              metrics=['accuracy'])

In [11]:
new_model.fit(X_train, Y_train, 
          batch_size=64, nb_epoch=10, verbose=2 )

new_model.save('model_V3.h5')

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/keras/models.py:942: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Epoch 1/10
 - 494s - loss: 0.7919 - acc: 0.5758
Epoch 2/10
 - 494s - loss: 0.6826 - acc: 0.6185
Epoch 3/10
 - 730s - loss: 0.6630 - acc: 0.6215
Epoch 4/10
 - 2461s - loss: 0.6310 - acc: 0.6469
Epoch 5/10
 - 2547s - loss: 0.6249 - acc: 0.6630
Epoch 6/10
 - 2503s - loss: 0.6119 - acc: 0.6698
Epoch 7/10
 - 2509s - loss: 0.6064 - acc: 0.6816
Epoch 8/10
 - 2507s - loss: 0.6121 - acc: 0.6888
Epoch 9/10
 - 2507s - loss: 0.5940 - acc: 0.6952
Epoch 10/10
 - 2577s - loss: 0.5852 - acc: 0.7019


In [ ]:
score = new_model.evaluate(X_test, Y_test, verbose=0)
print("Loss: " + str(score[0]))
print("Accuracy: " + str(score[1]))
new_model.save('model_V3.h5')

In [ ]:
from keras.models import model_from_json
model_json = new_model.to_json()
with open("newmodel.json","w") as json_file:
    json_file.write(model_json)
model.save_weights("newmodel.h5")
print("Saved model to disk")

# Defining the Model

In [22]:
model = Sequential()
# model.add(Convolution2D(32, 3, 3, activation='relu', input_shape=(img_size, img_size, 3)))
# model.add(MaxPooling2D(pool_size=(2,2)))
# model.add(Dropout(0.25))

# model.add(Convolution2D(32, 3, 3, activation='relu', input_shape=(img_size, img_size, 3)))
# model.add(MaxPooling2D(pool_size=(2,2)))
# model.add(Dropout(0.25))

# model.add(Convolution2D(32, 3, 3, activation='relu', input_shape=(img_size, img_size, 3)))
# model.add(MaxPooling2D(pool_size=(2,2)))
# model.add(Dropout(0.25))

model.add(Flatten(input_shape=(img_size, img_size, 3)))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))

adam = optimizers.SGD(lr=1e-4, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy',
              optimizer= adam,
              metrics=['accuracy'])


model.fit(X_train, Y_train, 
          batch_size=64, nb_epoch=10, verbose=2)

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/keras/models.py:942: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Epoch 1/10
 - 2s - loss: 0.7529 - acc: 0.5584
Epoch 2/10
 - 2s - loss: 0.6713 - acc: 0.6029
Epoch 3/10
 - 2s - loss: 0.6550 - acc: 0.6266
Epoch 4/10
 - 2s - loss: 0.6436 - acc: 0.6380
Epoch 5/10
 - 2s - loss: 0.6395 - acc: 0.6511
Epoch 6/10
 - 2s - loss: 0.6319 - acc: 0.6600
Epoch 7/10
 - 2s - loss: 0.6252 - acc: 0.6600
Epoch 8/10
 - 2s - loss: 0.6218 - acc: 0.6651
Epoch 9/10
 - 2s - loss: 0.6168 - acc: 0.6829
Epoch 10/10
 - 2s - loss: 0.6063 - acc: 0.6914


In [23]:
score = model.evaluate(X_test, Y_test, verbose=0)
print("Loss: " + str(score[0]))
print("Accuracy: " + str(score[1]))

Loss: 0.5872958221814758
Accuracy: 0.7394247044968323


# 3-Layer Convolution Model

In [8]:
model = Sequential()
model.add(Convolution2D(32, 3, 3, activation='relu', input_shape=(img_size, img_size, 3)))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Convolution2D(32, 3, 3, activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Convolution2D(64, 3, 3, activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
          
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))

adam = optimizers.SGD(lr=1e-4, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy',
              optimizer= adam,
              metrics=['accuracy'])


model.fit(X_train, Y_train, 
          batch_size=64, nb_epoch=10, verbose=2)

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:2: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), activation="relu", input_shape=(100, 100,...)`
  from ipykernel import kernelapp as app
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:5: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), activation="relu")`
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:8: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu")`
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/keras/models.py:942: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Epoch 1/10
 - 173s - loss: 0.6961 - acc: 0.5076
Epoch 2/10
 - 47s - loss: 0.6930 - acc: 0.5148
Epoch 3/10
 - 210s - loss: 0.6991 - acc: 0.4997
Epoch 4/10
 - 210s - loss: 0.6961 - acc: 0.5069
Epoch 5/10
 - 210s - loss: 0.6969 - acc: 0.4951
Epoch 6/10
 - 210s - loss: 0.6923 - acc: 0.5247
Epoch 7/10
 - 209s - loss: 0.6931 - acc: 0.5141
Epoch 8/10
 - 210s - loss: 0.6931 - acc: 0.5168
Epoch 9/10
 - 210s - loss: 0.6917 - acc: 0.5306
Epoch 10/10
 - 209s - loss: 0.6925 - acc: 0.5148


In [9]:
model.predict_proba(X_test, verbose=1)

169/169 [==============================] - 1s 8ms/step


array([[0.48978227, 0.5102178 ],
       [0.5126638 , 0.48733622],
       [0.48950937, 0.5104906 ],
       [0.49160662, 0.5083934 ],
       [0.46635497, 0.533645  ],
       [0.4912202 , 0.5087798 ],
       [0.49160662, 0.5083934 ],
       [0.48104823, 0.5189518 ],
       [0.48465884, 0.5153411 ],
       [0.50366956, 0.49633053],
       [0.49955177, 0.5004483 ],
       [0.49388328, 0.50611675],
       [0.49997947, 0.5000205 ],
       [0.48821166, 0.51178825],
       [0.49136847, 0.5086316 ],
       [0.47432464, 0.52567536],
       [0.4894852 , 0.51051474],
       [0.5095356 , 0.49046436],
       [0.492876  , 0.507124  ],
       [0.4995461 , 0.5004539 ],
       [0.5097288 , 0.49027118],
       [0.48297364, 0.51702636],
       [0.48454112, 0.5154589 ],
       [0.48896068, 0.5110393 ],
       [0.49964902, 0.50035095],
       [0.48273298, 0.51726705],
       [0.49384546, 0.50615454],
       [0.5086194 , 0.49138063],
       [0.47606763, 0.52393234],
       [0.49216706, 0.50783294],
       [0.

In [10]:
score = model.evaluate(X_test, Y_test, verbose=0)
print("Loss: " + str(score[0]))
print("Accuracy: " + str(score[1]))

Loss: 0.6892512585284442
Accuracy: 0.5798816568929063
